In [3]:
from dotenv import load_dotenv
import os
import pandas as pd
import cv2
from easyocr import Reader

In [2]:
# Initialisiere EasyOCR mit der gewünschten Sprache (in dem Fall Englisch)
reader = Reader(['en'], gpu=True)

# Create output list for individual adverts

In [3]:
# Definiere den Pfad zu deinem Zielordner
target_folder = "/home/giulia/Commercial-Brand-Differentiating-Message-Analysis/01_input_frames_all/2016/ADs_IG_2016/AD0417"


### Methode 2: Wasserzeichen abdecken mit schwarzen Box

In [4]:

# Funktion: Wasserzeichen-Bereich abdecken
def cover_watermark(frame, x, y, w, h, method="black"):
    """
    Überdeckt den Wasserzeichen-Bereich in einem Bild.
    Args:
    - frame: Das Bild/Frame (numpy-Array)
    - x, y: Obere linke Ecke des Wasserzeichen-Bereichs
    - w, h: Breite und Höhe des Wasserzeichen-Bereichs
    - method: Methode zum Abdecken ("black" oder "blur")
    """
    roi = frame[y:y+h, x:x+w]
    if method == "black":
        # Füllt den Bereich mit Schwarz
        frame[y:y+h, x:x+w] = (0, 0, 0)
    elif method == "blur":
        # Wendet einen Weichzeichner auf den Bereich an
        frame[y:y+h, x:x+w] = cv2.GaussianBlur(roi, (15, 15), 0)
    return frame


neue procee

In [5]:
def process_frames_with_watermark_removal(frames, x, y, w, h, method="black"):
    """
    Entfernt Wasserzeichen und führt OCR auf allen Frames durch, gruppiert die Ergebnisse nach Frames.
    Args:
        frames (List[Dict]): Eine Liste von Dictionaries mit Frame-Pfad und Werbungsnamen.
        x, y (int): Obere linke Ecke des Wasserzeichen-Bereichs.
        w, h (int): Breite und Höhe des Wasserzeichen-Bereichs.
        method (str): Methode zum Abdecken des Wasserzeichens ("black" oder "blur").

    Returns:
        pd.DataFrame: Ein DataFrame mit den gruppierten OCR-Ergebnissen (Spalten: Frame, Recognized_Text).
    """
    results_list = []

    # Iteriere durch alle Frames
    for frame_info in frames:
        ad_name = frame_info["ad"]
        frame_path = frame_info["frame_path"]

        # Lade das Bild
        image = cv2.imread(frame_path)

        # Abdeckung des Wasserzeichens
        image = cover_watermark(image, x, y, w, h, method)

        # Perform text recognition with EasyOCR (nur auf das abgedeckte Bild)
        ocr_results = reader.readtext(image)

        # Ergebnisse speichern
        for (bbox, text, prob) in ocr_results:
            # Überprüfe, ob der Text gültig ist (nicht leer oder nur Sonderzeichen)
            if text and text.strip() and len(text.strip()) > 1:  # Sicherstellen, dass der Text sinnvoll ist
                results_list.append({
                    "Frame": os.path.basename(frame_path),  # Nur der Dateiname
                    "Recognized_Text": text.strip(),       # Direkter Text ohne zusätzliche Filterung
                })

    # Ergebnisse in DataFrame umwandeln
    results_df = pd.DataFrame(results_list)

    # Gruppiere die Ergebnisse nach Frames
    merged_results = (
        results_df.groupby("Frame")
        .agg({
            "Recognized_Text": lambda x: ", ".join(x),  # Kombiniere alle erkannten Texte pro Frame
        })
        .reset_index()
    )

    return merged_results


### Ähnliche Frames Gruppieren, Duplikate entfernen

In [33]:
from collections import Counter
from rapidfuzz import fuzz
import enchant
from collections import Counter

# Enchant-Wörterbuch für Englisch
checker = enchant.Dict("en_US")
def count_correct_spelled_words(text):
    """Zählt, wie viele Wörter im Text laut Enchant korrekt geschrieben sind."""
    words = text.split()
    correct_count = 0
    for w in words:
        if checker.check(w):
            correct_count += 1
    return correct_count

# Funktion: Gruppiere Texte nach Ähnlichkeit und wähle den häufigsten
def remove_similar_texts(texts, threshold=60):
    text_groups = []
    # 1) Gruppiere nach Ähnlichkeit
    for text in texts:
        group_found = False
        for group in text_groups:
            if fuzz.ratio(text, group[0]) >= threshold:  # Überprüfe Ähnlichkeit
                group.append(text)
                group_found = True
                break
        if not group_found:
            text_groups.append([text])
    # 2) Wähle pro Gruppe den Text mit den meisten korrekt geschriebenen Wörtern
    #return [Counter(group).most_common(1)[0][0] for group in text_groups]
    best_texts = []
    for group in text_groups:
        # Für jeden Text in der Gruppe ermitteln wir die "Score" der korrekt geschriebenen Wörter
        best_text = max(group, key=lambda t: count_correct_spelled_words(t))
        best_texts.append(best_text)
    
    return best_texts


# Funktion: Sortiere die Frame-Spalte numerisch
def sort_frames(dataframe, frame_column="Frame"):
    """
    Sortiert die DataFrame basierend auf der AD-Nummer und der Frame-Nummer.
    Args:
        dataframe (pd.DataFrame): Die DataFrame mit der Frame-Spalte.
        frame_column (str): Der Name der Spalte, die sortiert werden soll.
    Returns:
        pd.DataFrame: Die sortierte DataFrame.
    """
    # Filtere Zeilen mit Texten, die weniger als 2 Zeichen haben
    dataframe = dataframe[dataframe["Recognized_Text"].str.len() >= 3].copy()

    # Konvertiere alle Werte der Spalte in Strings (falls nicht bereits)
    dataframe[frame_column] = dataframe[frame_column].astype(str)
    
    # Funktion zur Extraktion der AD-Nummer und der Frame-Nummer
    def extract_ad_and_frame(value):
        try:
            # Zerlege den Frame-Namen
            parts = value.split("_")
            # Extrahiere die Haupt-AD-Nummer
            ad_part = int(parts[0][2:])  # "AD0251" → 251
            # Extrahiere das Untersegment, wenn vorhanden
            sub_ad_part = int(parts[1]) if len(parts) > 2 and parts[1].isdigit() else 0
            # Extrahiere die Frame-Nummer
            frame_part = int(parts[2].split(".")[0])  # "Frame_250.png" → 250
            return (ad_part, sub_ad_part, frame_part)
        except (IndexError, ValueError):
            # Fehlerhafte Werte behalten die Haupt-AD-Nummer, aber kommen ans Ende innerhalb dieser Gruppe
            try:
                ad_part = int(value.split("_")[0][2:])  # Extrahiere nur die Haupt-AD-Nummer
                return (ad_part, float('inf'), float('inf'))
            except:
                # Wenn auch die Haupt-AD-Nummer nicht extrahiert werden kann, ans Ende schieben
                return (float('inf'), float('inf'), float('inf'))
    
    # Sortiere die DataFrame basierend auf AD-Nummer und Frame-Nummer
    dataframe = dataframe.sort_values(
        by=frame_column,
        key=lambda col: col.map(extract_ad_and_frame)
    )
    return dataframe

# Funktion: Erstelle die bereinigte Tabelle
def create_cleaned_dataframe(merged_results, ad_column="Frame", text_column="Recognized_Text", threshold=60):
    # Extrahiere den "Ad"-Namen aus den Frame-Namen
    merged_results["ad"] = merged_results[ad_column].apply(lambda x: x.split("_")[0])  
    # Gruppiere nach "ad" und bereinige Texte
    cleaned_data = []
    for ad, group in merged_results.groupby("ad"):
        unique_texts = remove_similar_texts(group[text_column].tolist(), threshold=threshold)
        unique_texts = [text for text in unique_texts if text.strip()]
        combined_text = "; ".join(unique_texts)  # Kombiniere die bereinigten Texte
        cleaned_data.append({"ad": ad, "recognized_text": combined_text})
    
    # Erstelle eine neue DataFrame
    cleaned_df = pd.DataFrame(cleaned_data)
    return cleaned_df 

In [ ]:
#cleaned_df.to_csv("cleaned_df_box.csv", index=False)

### durchführen

In [23]:
import pandas as pd

# Datei laden
file_path = "final_zwischenergebnis.csv"  # Pfad zur CSV-Datei
data = pd.read_csv(file_path)
df = pd.DataFrame(data)
df

,Frame,Recognized_Text,ad
0,AD0252_Frame_1610.png,Bravery:,AD0252
1,AD0252_Frame_1620.png,Bravery:,AD0252
2,AD0252_Frame_1630.png,Bravery:,AD0252
3,AD0252_Frame_1640.png,Bravery: It'5 what defines us.,AD0252
4,AD0252_Frame_1650.png,Bravery: It's what defines us.,AD0252
...,...,...,...
23657,AD0801_Frame_610.png,FN,AD0801
23658,AD0801_Frame_810.png,"Wea, ;om",AD0801
23659,AD0801_Frame_820.png,WeatherTeche_com,AD0801
23660,AD0801_Frame_830.png,WeatherTeche_com,AD0801


In [35]:
#1 sortieren
sorted_df = sort_frames(df,"Frame")
sorted_df

,Frame,Recognized_Text,ad
19,AD0252_Frame_730.png,"'903, Closed Course, obey speed ard traffic la...",AD0252
20,AD0252_Frame_740.png,"'3596, Closed course . Alays obey speed andd t...",AD0252
21,AD0252_Frame_750.png,"'3n, Closed course . Alays obey speed and traf...",AD0252
22,AD0252_Frame_760.png,Closec course . Always obey speed and traffic ...,AD0252
23,AD0252_Frame_770.png,Closec course . Alays obey speed and traffic |...,AD0252
...,...,...,...
23655,AD0801_Frame_520.png,"n=, Weai",AD0801
23658,AD0801_Frame_810.png,"Wea, ;om",AD0801
23659,AD0801_Frame_820.png,WeatherTeche_com,AD0801
23660,AD0801_Frame_830.png,WeatherTeche_com,AD0801


In [36]:
#2 
zwischenergebnis = create_cleaned_dataframe(sorted_df,text_column="Recognized_Text", threshold=50)
zwischenergebnis
zwischenergebnis.to_csv("test_zwischen.csv", index=False)

# Main Script

In [8]:
def load_all_frames(input_folder):
    """
    Listet alle Frames aus allen Unterordnern auf.
    Args:
        input_folder (str): Der Pfad zum Hauptordner mit den Frames.
    Returns:
        List[Dict]: Eine Liste von Dictionaries mit Frame-Pfad und zugehörigem Werbungsnamen.
    """
    frames = []
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith((".png", ".jpg")):  # Akzeptiere nur Bilddateien
                ad_name = os.path.basename(root)  # Name des Werbungs-Ordners
                frame_path = os.path.join(root, file)
                frames.append({"ad": ad_name, "frame_path": frame_path})
    return frames

In [9]:
def process_all_ads(input_folder, x=62, y=545, w=235, h=60, method="black", threshold=70):
    """
    Hauptpipeline, die alle Schritte für die Verarbeitung aller Werbungen durchführt.
    Args:
        input_folder (str): Der Pfad zum Hauptordner mit den Frames.
    Returns:
        pd.DataFrame: Ein bereinigtes DataFrame mit allen Werbungen und kombinierten Texten.
    """
    # Lade alle Frames
    frames = load_all_frames(input_folder)
    
    # Führe OCR auf allen Frames durch und entfernt Wasserzeichen
    ocr_df = process_frames_with_watermark_removal(frames, x, y, w, h, method)
    
    # df frames sortieren
    sorted_ocr_df = sort_frames(ocr_df,"Frame")

    #2 Bereinige und kombiniere die Texte
    final_ocr_df = create_cleaned_dataframe(sorted_ocr_df,text_column="Recognized_Text", threshold=50)

    # Daten mit Wörterbuch abgleichen und 
    final_ocr_df["cleaned_text"] = final_ocr_df["recognized_text"].apply(clean_recognized_text_with_spacy)
    
        
    return cleaned_df

In [10]:
input_folder = os.getenv("INPUT_FRAMES_ALL")  # Oder setze den Pfad direkt: "path/to/frames"

In [11]:
# Hauptskript
x, y, w, h = 62, 545, 235, 60  # Anpassen, falls nötig
method = "black"
threshold = 60

# 1. Lade alle Frames
frames = load_all_frames(input_folder)

print(f"Anzahl der geladenen Frames: {len(frames)}")
print(f"Anzahl der unterschiedlichen Werbungen: {len(set([frame['ad'] for frame in frames]))}")

# 2. Entferne Wasserzeichen und führe OCR aus
ocr_results_df = process_frames_with_watermark_removal(frames, x, y, w, h, method)


Anzahl der geladenen Frames: 64981
Anzahl der unterschiedlichen Werbungen: 537


In [15]:
ocr_results_df

,Frame,Recognized_Text,ad
0,AD0252_Frame_1610.png,Bravery:,AD0252
1,AD0252_Frame_1620.png,Bravery:,AD0252
2,AD0252_Frame_1630.png,Bravery:,AD0252
3,AD0252_Frame_1640.png,Bravery: It'5 what defines us.,AD0252
4,AD0252_Frame_1650.png,Bravery: It's what defines us.,AD0252
...,...,...,...
23657,AD0801_Frame_610.png,FN,AD0801
23658,AD0801_Frame_810.png,"Wea, ;om",AD0801
23659,AD0801_Frame_820.png,WeatherTeche_com,AD0801
23660,AD0801_Frame_830.png,WeatherTeche_com,AD0801


In [ ]:

# 3. Bereinige die Ergebnisse und kombiniere Texte
final_results_df = create_cleaned_dataframe(ocr_results_df, text_column="Recognized_Text", threshold=60)

# 4. Zeige die Ergebnisse
from IPython.display import display
display(final_results_df)


,ad,recognized_text
0,AD0252,Bravery:; Bravery: It'5 what defines us.; Audi...
1,AD0253,"AFF, MEEP; Ficlionalizalion; Acllowallzallou; ..."
2,AD0254,"soluor CNRAL, Sprint, #Mobile:, atet; MCINTRAL..."
3,AD0255,Screen images suulal; 112z8; Fuzt;; 1e04213; s...
4,AD0256,"BECKS, SAPPHII; bEck:, ALC /Vol, BECKS, SAPPHI..."
...,...,...
530,AD0797,"Uber, Eats; Foil, Aluminum; Prop food Do not e..."
531,AD0798,"verizon, 12tia007; Signupat verizon com/SG, 5G..."
532,AD0799,"3i, Go, Le ta, (STop, talae, Wae, Weun; (STop,..."
533,AD0800,"20,2012. Seth T. was struck by lightning:, May..."


Ergebnisse wurden erfolgreich in 'final_ocr_results.csv' gespeichert.


### ENCHANT für Rechtschreibkorrektur + NER Modell für Erkennung von Markenname und Produktname

In [46]:
import re
import enchant

# Wörterbuch für Englisch
english_dict = enchant.Dict("en_US")


In [41]:
# Funktion, um zu prüfen, ob ein Wort korrekt ist
def is_word_correct(word):
    return checker.check(word)

# Beispielwörter
words = ["correct", "speling", "WiFi", "ridgeline","isn't","Don't","we'll","won't"]

# Ergebnis prüfen
for word in words:
    print(f"{word}: {'richtig' if is_word_correct(word) else 'falsch'}")

correct: richtig
speling: falsch
WiFi: richtig
ridgeline: falsch
isn't: richtig
Don't: richtig
we'll: richtig
won't: richtig


In [39]:
# Textverarbeitung
text = "Audi Honda Ebay Axes introduces the new Ridgeline RTL in 2024 Ridgeline isn't."
doc = nlp(text)

# Entitäten anzeigen
for ent in doc.ents:
    print(f"Text: {ent.text}, Typ: {ent.label_}")

Text: Audi, Typ: ORG
Text: Honda, Typ: ORG
Text: Ebay Axes, Typ: ORG
Text: Ridgeline RTL, Typ: FAC
Text: 2024, Typ: CARDINAL
Text: Ridgeline, Typ: GPE


### mit Spacy

In [53]:
import re
import enchant
import spacy

# Wörterbuch für Englisch
english_dict = enchant.Dict("en_US")

# Lade spaCy-Modell
nlp = spacy.load("en_core_web_md")

def clean_recognized_text_with_spacy(text):
    """
    Bereinigt erkannte Texte und prüft Wörter auf ihre Gültigkeit basierend auf einem Wörterbuch (enchant)
    und Named Entity Recognition (NER) mit spaCy.
    
    Args:
        text (str): Eingabetext (eine lange Zeichenkette).
        
    Returns:
        str: Bereinigter Text.
    """
    # 1. Bindestriche durch Leerzeichen ersetzen. Sonderzeichen entfernen, nur alphanumerische Zeichen und Leerzeichen behalten
    cleaned_text0 = re.sub(r"-", " ", text)
    cleaned_text1 = re.sub(r"[^\w\s']|(?<=\s)'|'(?=\s)", "", cleaned_text0)
    cleaned_text = re.sub(r"(?<=\w)'(?!\w)", "", cleaned_text1)
    print("Bereinigter Text ohne Sonderzeichen:", cleaned_text)

    # 2. Text in Wörter aufteilen und in Kleinbuchstaben umwandeln
    words_in_text = cleaned_text.split()
    print("Aufgeteilte Wörter (Kleinschreibung):", words_in_text)

    # 3. spaCy Named Entity Recognition (NER)
    doc = nlp(cleaned_text)
    recognized_entities = {ent.text.lower() for ent in doc.ents}
    print("Von spaCy erkannte Entitäten:", recognized_entities)

    # 4. Filtere Wörter basierend auf Wörterbuch und spaCy-Entitäten
    meaningful_words = [
        word for word in words_in_text
        if english_dict.check(word) or word.lower() in recognized_entities
    ]

    # 5. Bereinige den Text, indem nur relevante Wörter beibehalten werden
    final_cleaned_text = " ".join(meaningful_words)
    return final_cleaned_text


In [38]:
text = "isn't 903, Closed Course, obey speed ard traffic laws;, Aluys; PRINCIPAL, ONLY; PRINCE; PathctpaE, BHrSoN, ISQHIOOL, Hu; Bravery:; Bravery: It'5 what defines us.; Audi, CW), Truth in Engineering, #BraveryWins; SCORING SUMMARY, BALTIMORE, PLAYS 6, Art, YARDS 51, RAVENS, TIME 2:29, Jin: 13-YARD TD RECEPTION"

print(clean_recognized_text_with_spacy(text))

Bereinigter Text ohne Sonderzeichen: isnt 903 Closed Course obey speed ard traffic laws Aluys PRINCIPAL ONLY PRINCE PathctpaE BHrSoN ISQHIOOL Hu Bravery Bravery It5 what defines us Audi CW Truth in Engineering BraveryWins SCORING SUMMARY BALTIMORE PLAYS 6 Art YARDS 51 RAVENS TIME 229 Jin 13 YARD TD RECEPTION
Aufgeteilte Wörter (Kleinschreibung): ['isnt', '903', 'Closed', 'Course', 'obey', 'speed', 'ard', 'traffic', 'laws', 'Aluys', 'PRINCIPAL', 'ONLY', 'PRINCE', 'PathctpaE', 'BHrSoN', 'ISQHIOOL', 'Hu', 'Bravery', 'Bravery', 'It5', 'what', 'defines', 'us', 'Audi', 'CW', 'Truth', 'in', 'Engineering', 'BraveryWins', 'SCORING', 'SUMMARY', 'BALTIMORE', 'PLAYS', '6', 'Art', 'YARDS', '51', 'RAVENS', 'TIME', '229', 'Jin', '13', 'YARD', 'TD', 'RECEPTION']
Von spaCy erkannte Entitäten: {'51', '903', 'audi cw truth', 'isqhiool hu bravery bravery', 'baltimore', '229', 'ravens time'}
903 Closed Course obey speed traffic laws PRINCIPAL ONLY PRINCE Bravery Bravery what defines us Audi CW Truth in Eng

In [55]:
zwischenergebnis["cleaned_text"] = zwischenergebnis["recognized_text"].apply(clean_recognized_text_with_spacy)

#Speichere die Ergebnisse
zwischenergebnis.to_csv("test_ocr_to_merge.csv", index=False)
print("Ergebnisse wurden erfolgreich in 'ocr_to_merge.csv' gespeichert.")

Bereinigter Text ohne Sonderzeichen: '3n Closed course  Alays obey speed and traffic aws PRINCIPAL ONLY PRINCE PathctpaE BHrSoN ISQHIOOL Hu Bravery Bravery It's what defines us Audi CW Truth in Engineering BraveryWins Away Fiel 22 SCORING SUMMARY BALTIMORE ANQUAN BOLDIN AtH REC TD THIS POSTSEASON MFD Art 4 REC TD IN REG SEASON RAVENS
Aufgeteilte Wörter (Kleinschreibung): ["'3n", 'Closed', 'course', 'Alays', 'obey', 'speed', 'and', 'traffic', 'aws', 'PRINCIPAL', 'ONLY', 'PRINCE', 'PathctpaE', 'BHrSoN', 'ISQHIOOL', 'Hu', 'Bravery', 'Bravery', "It's", 'what', 'defines', 'us', 'Audi', 'CW', 'Truth', 'in', 'Engineering', 'BraveryWins', 'Away', 'Fiel', '22', 'SCORING', 'SUMMARY', 'BALTIMORE', 'ANQUAN', 'BOLDIN', 'AtH', 'REC', 'TD', 'THIS', 'POSTSEASON', 'MFD', 'Art', '4', 'REC', 'TD', 'IN', 'REG', 'SEASON', 'RAVENS']
Von spaCy erkannte Entitäten: {'ravens', 'baltimore anquan boldin', '22', '4', 'audi cw truth', 'isqhiool hu bravery bravery', "'3n"}
Bereinigter Text ohne Sonderzeichen: NFF WE

In [54]:
texts = [
    "Owner' of the car",
    "Owner's rights",
    "This is a test, and we'll see if it's working.",
    "Special chars like #, $, and % should be gone!",
    "Don't forget John's car isn't here."
]

# Anwendung auf die Beispieltexte
for text in texts:
    result = clean_recognized_text_with_spacy(text)
    print(f"Original: {text}")
    print(f"Gereinigt: {result}")
    print()

Bereinigter Text ohne Sonderzeichen: Owner of the car
Aufgeteilte Wörter (Kleinschreibung): ['Owner', 'of', 'the', 'car']
Von spaCy erkannte Entitäten: set()
Original: Owner' of the car
Gereinigt: Owner of the car

Bereinigter Text ohne Sonderzeichen: Owner's rights
Aufgeteilte Wörter (Kleinschreibung): ["Owner's", 'rights']
Von spaCy erkannte Entitäten: {'owner'}
Original: Owner's rights
Gereinigt: Owner's rights

Bereinigter Text ohne Sonderzeichen: This is a test and we'll see if it's working
Aufgeteilte Wörter (Kleinschreibung): ['This', 'is', 'a', 'test', 'and', "we'll", 'see', 'if', "it's", 'working']
Von spaCy erkannte Entitäten: set()
Original: This is a test, and we'll see if it's working.
Gereinigt: This is a test and we'll see if it's working

Bereinigter Text ohne Sonderzeichen: Special chars like   and  should be gone
Aufgeteilte Wörter (Kleinschreibung): ['Special', 'chars', 'like', 'and', 'should', 'be', 'gone']
Von spaCy erkannte Entitäten: set()
Original: Special chars